# Cálculo de Energia Livre usando VASP e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/elvis/Programs/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

# Importando o VASP calculator do ASE
from ase.calculators.vasp import Vasp

from ase import Atoms
from ase.io import write, read
from ase.visualize import view

import nglview as nv

import numpy as np

## Cálculo dos Modos Vibracionais 

In [ ]:
# 1. Define H2O molecule
h2o = Atoms('H2O',
            positions=[[0.000, 0.000, 0.000],
                       [0.757, 0.586, 0.000],
                       [-0.757, 0.586, 0.000]])

h2o.center(vacuum=5.0)  # Center the molecule in the cell
h2o.pbc = True

### Passo 1: Otimização da Geometria da Molécula 

In [ ]:
# 2. Geometry optimization first
calc_relax = Vasp(
    directory='h2o_opt',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=h2o
)
calc_relax.calculate(h2o)

Demorou 48 s

In [ ]:
E_0_h2o = h2o.get_potential_energy()

print(f'Optimized energy of H2O: {E_0_h2o} eV')

### Passo 2: Cálculo de DFTPT para Hessiana

In [ ]:
# 3. dftpt vibrational calculation (uses linear response)
calc_vib = Vasp(
    directory='h2o_dftpt',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=8,           # dftpt for vibrational analysis
    nfree=2,            # Central difference (default)
    lepsilon=True,      # Compute Born charges and dielectric tensor
    lreal=False,
    lwave=False, lcharg=False,
    atoms=h2o
)

calc_vib.calculate(h2o)

Demorou 2m 43 s

### Passo 3: Cálculo das frequências e das energias a partir da Hessiana

Re-lendo cálculo já realizado

In [ ]:
calc_vib = Vasp(restart=True,directory='h2o_dftpt')

Lendo os dados de vibração a partir da matriz Hessiana

In [ ]:
vib_water = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_water.get_frequencies()

# All displacement vectors (n_modes, n_atoms, 3)
modes = vib_water.get_modes()

for i, f in enumerate(freqs):
    # write frequencies in cm-1 separeting the real and imaginary parts
    print(f"Mode {i:01d}: {f.real:.2f} cm-1 (real), {f.imag:.2f} cm-1 (imaginary)")

### Passo 4 (opcional): Exporta os arquivos de trajetória dos modos vibracionais 

In [ ]:
# Get equilibrium geometry 
atoms_eq = vib_water.get_atoms()

# --- Loop over all modes to generate trajectories ---
amp = 0.2           # Å: visualization amplitude (not physical)
n_cycles = 5        # number of oscillations you want
n_frames = 50      # total number of frames (smooth animation)

# --- Create output directory ---
directory = "water_vib_modes"
os.makedirs(directory, exist_ok=True)

for i, f in enumerate(freqs):
    # Skip imaginary and near-zero modes
    if f.imag != 0 or abs(f.real) < 50.0:
        continue

    mode = modes[i]
    frames = []
    for j in range(n_frames):
        phase = 2 * np.pi * n_cycles * j / n_frames
        atoms = atoms_eq.copy()
        atoms.positions += amp * np.sin(phase) * mode
        frames.append(atoms)

    fname = directory+f"/mode_{i:01d}.traj"
    write(fname, frames)
    print(f"Mode {i:01d}: {f.real:.2f} cm-1 = {f.real*0.12398:.2f} meV")

print(f"✅ All vibrational modes exported in {directory}")

In [ ]:
# Load the vibrational trajectory file
traj = read(directory+'/mode_6.traj', index=':')

# Create NGLView widget
w = nv.show_asetraj(traj)
w.add_ball_and_stick()

w

## Cálculo de ZVPE

### Passo 1: Calcula as energias vibracionais a partir das frequências de vibração

$ E = h \nu$

In [ ]:
# convert cm^{-1} to eV
h = 4.1356675e-15 # eV*s
c = 3.0e10 #cm/s
vib_energies_h2o = []
for i,f in enumerate(freqs):
    if f.imag != 0 or abs(f.real) < 50.0:
        continue

    e = h*c*f.real
    vib_energies_h2o.append(e)
    print(f'Mode {i:01d}: {e:<.3f} eV')

### Passo 2: Calcula o ZPVE
$ZVPE = \sum_{v=1}^{\text{modos de vibração}} \frac{1}{2} h \nu_v$

In [ ]:
ZPE_h2o = 0.5 * sum(vib_energies_h2o)
print(f'Zero-point energy (ZPE) of H2O: {ZPE_h2o:.3f} eV')

## Cálculo de Qtds Termodinâmicas

Usando função `IdealGasThermo` do `ASE` facilita nossa vida!

In [ ]:
from ase.thermochemistry import IdealGasThermo

h2o_thermo = IdealGasThermo(vib_energies=vib_energies_h2o,
                        potentialenergy=E_0_h2o, atoms=h2o,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

`symmetrynumber`: Table 10.1 and
        Appendix B of C. Cramer "Essentials of Computational Chemistry",
        2nd Ed

In [ ]:
# temperature in K and pressure in Pa
G_h2o = h2o_thermo.get_gibbs_energy(temperature=298.15, pressure=1e5)

> DPL: Diversão para o Lar
> 
> 📝 Repita os procedimentos acima agora para a molécula de CO_2.
>  
> Atente para a geometria linear da molécula e seu grupo de simetria. 
> 
> 